tf.keras でcifar10 train

In [2]:
!pwd
import sys
sys.executable

/c/Users/shingo/jupyter_notebook/tfgpu_py36_work/02_keras_py/experiment/01_code_test


'C:\\Users\\shingo\\Anaconda3\\envs\\tfgpu20\\python.exe'

In [1]:
from tensorflow import keras 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()
y_train, y_test = keras.utils.to_categorical(y_train), keras.utils.to_categorical(y_test)
train_gen = ImageDataGenerator(rescale=1.0/255, rotation_range=18, zoom_range=0.15).flow(X_train, y_train, batch_size=16)
valid_gen = ImageDataGenerator(rescale=1.0/255).flow(X_test, y_test, batch_size=16, shuffle=False)

input_tensor = keras.layers.Input(shape=(32, 32, 3))
model = keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
x = model.output
x = keras.layers.GlobalAveragePooling2D(name='FC_avg')(x)
predictions = keras.layers.Dense(10, activation='softmax', name='pred')(x)
model = keras.models.Model(inputs=model.input, outputs=predictions)
model.compile(loss=keras.backend.categorical_crossentropy, optimizer='sgd', metrics=['accuracy'])

model.fit_generator(train_gen
                    , steps_per_epoch=X_train.shape[0]//16
                    , epochs=2
                    , validation_data=valid_gen
                    , validation_steps=X_test.shape[0]//16
                    , verbose=1)

Epoch 1/2
3125/3125 [==============================] - 174s 56ms/step - loss: 0.9887 - accuracy: 0.6550 - val_loss: 0.6401 - val_accuracy: 0.7808
Epoch 2/2
3125/3125 [==============================] - 165s 53ms/step - loss: 0.6225 - accuracy: 0.7855 - val_loss: 0.6467 - val_accuracy: 0.7723


In [3]:
# モジュールimport
import os, sys
sys.path.append(r'C:\Users\shingo\jupyter_notebook\tfgpu_py36_work\02_keras_py')

from dataset import plot_log, prepare_data, util

from transformer import tf_get_train_valid_test as get_train_valid_test
from transformer import my_generator

from model import tf_define_model as define_model
from model import tf_multi_loss as multi_loss
from model import tf_my_callback as my_callback
from model import my_metric

from predicter import roc_curve, conf_matrix, ensemble_predict
from predicter import tf_grad_cam as grad_cam
from predicter import tf_base_predict as  base_predict

%reload_ext autoreload
%autoreload 2

In [8]:
def get_class_fine_tuning_parameter_suggestions(trial) -> dict:
    """
    Get parameter sample for class fine_tuning (like Keras)
    Args:
        trial(trial.Trial):
    Returns:
        dict: parameter sample generated by trial object
    """
    return {
        'gpu_count': 1,
        'img_rows': 331,
        'img_cols': 331,
        'channels': 3,
        'batch_size': 16,
        'classes': ['negative', 'positive'],
        'num_classes': 2,
        'train_data_dir': './input/CV1_small/train',
        'validation_data_dir': './input/CV1_small/validation',
        'class_mode': 'categorical', # generatorのラベルをone-hotベクトルに変換する場合。generatorのラベルを0か1のどちらかに変えるだけなら'binary'
        'output_dir': r'D:\work\02_keras_py\experiment\01_code_test\output_test\tmp',
        'activation': 'softmax',
        'metrics': ['accuracy'],
        'model_path': None,
        'num_epoch': 50,
        'n_multitask': 1, 
        #'multitask_pred_n_node'=2 # マルチタスクの各クラス数
        # model param
        'choice_model': trial.suggest_categorical('choice_model', ['InceptionV3', 'EfficientNet']),
        'fcpool': trial.suggest_categorical('fcpool', ['attention', 'GlobalAveragePooling2D']),
        'is_skip_bn': trial.suggest_categorical('is_skip_bn', [True, False]),
        'trainable': trial.suggest_categorical('trainable', ['all', 100, 200, 300, 400, 500, 600]),
        'efficientnet_num': trial.suggest_categorical('efficientnet_num', [3,7]),
        # full layer param
        'fcs': trial.suggest_categorical('fcs', [[], [100], [256], [512, 256], [1024, 512, 256]]),
        'drop': trial.suggest_categorical('drop', [0.3, 0.5, 0.7]),
        'is_add_batchnorm': trial.suggest_categorical('is_add_batchnorm', [True, False]),
        'l2_rate': trial.suggest_categorical('l2_rate', [1e-5, 5e-5, 1e-4, 5e-4, 1e-3]),
        # optimizer param
        'choice_optim': trial.suggest_categorical('choice_optim', ['sgd', 'adadelta', 'adam', 'adamax', 'nadam', 'adabound']),
        'lr': trial.suggest_categorical('lr', [1e-3, 1e-2, 1e-1]),
        'decay': trial.suggest_categorical('decay', [0.0, 1e-6, 1e-5, 1e-4]) # 各更新上の学習率減衰
    }

def get_ImageDataGenerator_parameter_suggestions(trial) -> dict:
    """
    Get parameter sample for ImageDataGenerator (like Keras)
    Args:
        trial(trial.Trial):
    Returns:
        dict: parameter sample generated by trial object
    """
    return {
        'rescale': 1.0/255.0,
        'width_shift_range': trial.suggest_categorical('height_shift_range', [0.0, 0.25]),
        'height_shift_range': trial.suggest_categorical('height_shift_range', [0.0, 0.25]),
        'horizontal_flip': trial.suggest_categorical('horizontal_flip', [True, False]),
        'vertical_flip': trial.suggest_categorical('vertical_flip', [True, False]),
        'shear_range': trial.suggest_categorical('shear_range', [0.0, 20, 50]),
        'zoom_range': trial.suggest_categorical('zoom_range', [0.0, 0.2, 0.5]),
        'rotation_range': trial.suggest_categorical('rotation_range', [0.0, 45, 60, 90]),
        'channel_shift_range': trial.suggest_categorical('channel_shift_range', [0.0, 100, 200]),
        'brightness_range': trial.suggest_categorical('brightness_range', [[1.0, 1.0], [0.3, 1.0]]),
        # MyImageDataGenerator param
        'random_erasing_prob': trial.suggest_categorical('random_erasing_prob', [0.0, 0.5]),
        'random_erasing_maxpixel': 255.,
        'mix_up_alpha': trial.suggest_categorical('mix_up_alpha', [0.0, 0.2]),
        'ricap_beta': trial.suggest_categorical('ricap_beta', [0.0, 0.3]),
        'is_base_aug': trial.suggest_categorical('is_base_aug', [False]),
        'is_grayscale': trial.suggest_categorical('is_grayscale', [False]),
    }

def get_Augmentor_parameter_suggestions(trial) -> dict:
    """
    Get parameter sample for Augmentor
    Args:
        trial(trial.Trial):
    Returns:
        dict: parameter sample generated by trial object
    """
    return {
        'rescale': 1.0/255.0,
        'rotate90': trial.suggest_categorical('rotate90', [0.0, 0.5]),
        'rotate180': trial.suggest_categorical('rotate180', [0.0, 0.5]),
        'rotate270': trial.suggest_categorical('rotate270', [0.0, 0.5]),
        'rotate_prob': trial.suggest_categorical('rotate_prob', [0.0, 0.5]),
        'rotate_max_left': trial.suggest_categorical('rotate_max_left', [20, 60, 90]),
        'rotate_max_right': trial.suggest_categorical('rotate_max_right', [20, 60, 90]),
        'crop_prob': trial.suggest_categorical('crop_prob', [0.0, 0.5]),
        'crop_area': trial.suggest_categorical('crop_area', [0.8, 0.5]),
        'crop_by_size_prob': trial.suggest_categorical('crop_by_size_prob', [0.0, 0.5]),
        'crop_by_width': trial.suggest_categorical('crop_by_width', [224]),
        'crop_by_height': trial.suggest_categorical('crop_by_height', [224]),
        'crop_by_centre': trial.suggest_categorical('crop_by_centre', [True, False]),
        'shear_prob': trial.suggest_categorical('shear_prob', [0.0, 0.5]),
        'shear_magni': trial.suggest_categorical('shear_magni', [20, 50]),
        'skew_prob': trial.suggest_categorical('skew_prob', [0.0, 0.5]),
        'skew_magni': trial.suggest_categorical('skew_magni', [20, 50]),
        'zoom_prob': trial.suggest_categorical('zoom_prob', [0.0, 0.5]),
        'zoom_min': trial.suggest_categorical('zoom_min', [0.2, 0.5, 0.9]),
        'zoom_max': trial.suggest_categorical('zoom_max', [1.2, 1.5, 1.9]),
        'flip_left_right': trial.suggest_categorical('flip_left_right', [0.0, 0.5]),
        'flip_top_bottom': trial.suggest_categorical('flip_top_bottom', [0.0, 0.5]),
        'random_erasing_prob': trial.suggest_categorical('random_erasing_prob', [0.0, 0.5]),
        'random_erasing_area': trial.suggest_categorical('random_erasing_area', [0.3]),
        'random_dist_prob': trial.suggest_categorical('random_dist_prob', [0.0, 0.5]),
        'random_dist_grid_width': trial.suggest_categorical('random_dist_grid_width', [4]),
        'random_dist_grid_height': trial.suggest_categorical('random_dist_grid_height', [4]),
        'random_dist_grid_height': trial.suggest_categorical('random_dist_grid_height', [4]),
        'random_dist_magnitude': trial.suggest_categorical('random_dist_magnitude', [8]),
        'black_and_white': trial.suggest_categorical('black_and_white', [0.0, 0.5]),
        'greyscale': trial.suggest_categorical('greyscale', [0.0, 0.5]),
        'invert': trial.suggest_categorical('invert', [0.0, 0.5])
    }

import optuna

def objective(trial):
    trial_class = get_class_fine_tuning_parameter_suggestions(trial)
    trial_imagegen = get_ImageDataGenerator_parameter_suggestions(trial)
    trial_augmentor = get_Augmentor_parameter_suggestions(trial)
    trial_merge = {**trial_class, **trial_imagegen, **trial_augmentor}
    #display(trial_merge)
    print(trial_merge['invert'])
    #define_model.get_fine_tuning_model(**trial)
    
study = optuna.create_study(direction='minimize') # 最小化
study.optimize(objective, n_trials=5)

0.0


[W 2020-01-13 22:29:08,773] Setting status of trial#0 as TrialState.FAIL because the returned value from the objective function cannot be casted to float. Returned value is: None


0.0


[W 2020-01-13 22:29:09,977] Setting status of trial#1 as TrialState.FAIL because the returned value from the objective function cannot be casted to float. Returned value is: None


0.5


[W 2020-01-13 22:29:11,215] Setting status of trial#2 as TrialState.FAIL because the returned value from the objective function cannot be casted to float. Returned value is: None


0.5


[W 2020-01-13 22:29:12,468] Setting status of trial#3 as TrialState.FAIL because the returned value from the objective function cannot be casted to float. Returned value is: None


0.0


[W 2020-01-13 22:29:13,756] Setting status of trial#4 as TrialState.FAIL because the returned value from the objective function cannot be casted to float. Returned value is: None
